## Example of usage of corn fertilization environment

Here, we present an example of how to use the basic corn fertilization environment to simulate an expert fertilizaiton policy based on the [recommendations by the Pennsylvania state university](https://extension.psu.edu/nitrogen-fertilization-of-corn). The simulation will be for the years 1980 and 1981 with fixed weather from Rock Springs, PA.

In [ ]:
from cyclesgym.envs import Corn
from cyclesgym.envs.weather_generator import FixedWeatherGenerator
from cyclesgym.utils.paths import CYCLES_PATH  # Path to Cycles directory
from cyclesgym.managers import OperationManager
from cyclesgym.policies.dummy_policies import OpenLoopPolicy

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

First, we want to define the default operations, i.e., those operations that we want to perform but that are not controlled by the RL agents (e.g., irrigation, tillage, planting, and fertilization of nutrients different from N). We do this by loading and inspecting an existing operation file.

In [ ]:
op_manager = OperationManager(CYCLES_PATH.joinpath('input', 'ContinuousCorn.operation'))
print(op_manager)

We can see there are 3 operations: N fertilization, planting, and tillage. N fertilization will be zeroed out automatically not to interfere with the decisions of the RL agents. 

Let's define a temporary operation file without tillage to see how one can specify custom default operations.

Internally, the operation manager stores all the operations in a dictionary with keys of the type (YEAR, DOY, OP_TYPE). We can provide a list of such keys to the `delete_operations` method of the manager to remove the desired operations.

In [ ]:
# Delete the operation
op_manager.delete_operations([(1, 110, 'TILLAGE')])
print(op_manager)

# Save the new operation file to make it available to Cycles
new_op_path = CYCLES_PATH.joinpath('input', 'ContinuousCornNoTillageExample.operation')
op_manager.save(new_op_path)

Now, we can initialize the environment.

In [ ]:
env = Corn( 
    delta=7,                # Time step is one week
    n_actions=11,           # 11 actions available between [0, maxN]   
    maxN=150,
    operation_file=new_op_path.name,      # Using the default operation we defined in the previous steps
    weather_generator_class=FixedWeatherGenerator,   # Deterministic weather
    weather_generator_kwargs={'base_weather_file': CYCLES_PATH.joinpath('input', 'RockSprings.weather')},   # Location of the weather data
    start_year=1980,
    end_year=1981)

Let' reset it and take a look at the observations availalble in this environment. 

In [ ]:
obs = env.reset()
for o, o_name in zip(obs, env.observer.obs_names):
    print(f'Variable name: {o_name:<15}\tValue: {o}')


Now we specify the expert policy. This expert policy is open loop in the sense that that it does not take into account any information about the system and the action applied only depends on the day or the year. As a consequence, it can be expressed as a pre-specified sequence of actions rather than as a mapping from observations to actions. In particular, this consits in applying 150kg/ha around the planting date.

In [ ]:
# Initalize as no fertilization on any week.
action_sequence = [0] * 105

# At the beginning of the 16th week (day of the year 16*7=112) apply action 10, i.e., the max action in the set 
# {0, 1, ..., 10}. Given how our environment was defined, this corresponds to 150kg/ha.
action_sequence [15] = 10  

# Same for the second year
action_sequence[68] = 10

pi = OpenLoopPolicy(action_sequence)  # Provides an interface for open loop policies that is the same as the stable baselines interface

Now we can simulate the system and collect info about the policy, the reward, and information about leaching, volatilization, and emission.

In [ ]:
actions = np.zeros(105, dtype=float)
rewards = np.zeros(105, dtype=float)
leaching = np.zeros(105, dtype=float)
volatilization = np.zeros(105, dtype=float)
emission = np.zeros(105, dtype=float)


obs = env.reset()
week = 0

while True:
    a, _ = pi.predict(obs)
    obs, r, done, info = env.step(a)
    actions[week] = a
    rewards[week] = r
    leaching[week] = info['cost_leaching']
    volatilization[week] = info['cost_volatilization']
    emission[week] = info['cost_emission']
    week += 1
    if done:
        break


Now, we can plot all the information we collected.

In [ ]:
for data, name in zip([actions, rewards, leaching, volatilization, emission], 
                      ['Actions', 'Rewards', 'Leaching', 'Volatilization', 'Emission']):
    plt.figure()
    plt.plot(data)
    plt.title(name)
    plt.xlabel('Weeks')



In [ ]:
# Remove temporary file used for this example.
new_op_path.unlink()